# `nanoGPT`: GPT-2 Large (775M Params)

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

python3 -c 'import ngpt; print(ngpt.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has ngpt installed. Nothing to do."
else
    echo "Does not have ngpt installed. Installing..."
    git clone 'https://github.com/saforem2/nanoGPT'
    python3 nanoGPT/data/shakespeare_char/prepare.py
    python3 -m pip install -e nanoGPT -vvv
fi

/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py
Has ngpt installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from enrich import get_logger
log = get_logger('jupyter')
log.info(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2023-11-30 07:43:58][INFO][3434626787.py:7] - /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config, PROJECT_ROOT
from ngpt.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['MASTER_PORT'] = '5631'
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=owt',
        'model=gpt2_large',
        'model.block_size=128',
        'optimizer=gpt2_large',
        'train=gpt2_large',
        'train.init_from=gpt2-large',
        'train.max_iters=1000',
        'train.dtype=bfloat16',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu23
  Local device: mlx5_0
--------------------------------------------------------------------------


[2023-11-30 07:44:02][INFO][configs.py:72] - Setting HF_DATASETS_CACHE to /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/.cache/huggingface/datasets
Failed to download font: Source Sans Pro, skipping!
Failed to download font: Titillium WebRoboto Condensed, skipping!
[2023-11-30 07:44:05][INFO][configs.py:295] - Loading val from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/openwebtext/val.bin
[2023-11-30 07:44:05][INFO][configs.py:295] - Loading train from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/openwebtext/train.bin
[2023-11-30 07:44:05][WARNING][configs.py:330] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-30 07:44:05][INFO][configs.py:270] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-30 07:44:05][INFO][configs.py:432] - Tokens per iteration: 512
[2023-11-30 07:44:05][INFO][configs.py:454] - Using <torch.amp.autocast_mode.autocas

## Prompt (prior to training)

In [4]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-30 07:44:53][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-30 07:44:53][INFO][1657463709.py:4] - ['response']:

What is a supercomputer? A supercomputer is a computer that can run more than one hundred thousand instructions per second. It's basically like a computer that can simulate a human brain. So, if we had a supercomputer with a billion instructions per second, that's forty-seven times as fast as the brain itself.

A supercomputer is something that we can invent and code and build. But we're not going to have one, because we're not going to have a supercomputer. There are some things that we can do with computers, that computers can't accomplish.

So, let's think about, for instance, programming language. We have ten thousand programming languages. So, what is a programming language? A programming language is a language that lets you write a program. And if you say, "OK, if I want to do this, I'm going to write a program to do this," you have a programming lang

## Train

In [5]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

[2023-11-30 07:46:25][INFO][trainer.py:518] - step=100 loss=2.422 dt=563.654 sps=1.774 mtps=0.001 mfu=36.352 train_loss=2.662 val_loss=2.654
[2023-11-30 07:47:23][INFO][trainer.py:518] - step=200 loss=2.877 dt=575.121 sps=1.739 mtps=0.001 mfu=36.280 train_loss=2.662 val_loss=2.654
[2023-11-30 07:48:22][INFO][trainer.py:518] - step=300 loss=2.465 dt=586.089 sps=1.706 mtps=0.001 mfu=36.148 train_loss=2.662 val_loss=2.654
[2023-11-30 07:49:20][INFO][trainer.py:518] - step=400 loss=2.615 dt=580.777 sps=1.722 mtps=0.001 mfu=36.061 train_loss=2.662 val_loss=2.654
[2023-11-30 07:50:19][INFO][trainer.py:518] - step=500 loss=3.006 dt=589.075 sps=1.698 mtps=0.001 mfu=35.933 train_loss=2.662 val_loss=2.654
[2023-11-30 07:51:17][INFO][trainer.py:518] - step=600 loss=2.524 dt=585.214 sps=1.709 mtps=0.001 mfu=35.841 train_loss=2.662 val_loss=2.654
[2023-11-30 07:52:16][INFO][trainer.py:518] - step=700 loss=2.904 dt=600.498 sps=1.665 mtps=0.001 mfu=35.669 train_loss=2.662 val_loss=2.654
[2023-11-30 0

## Evaluate Model

In [7]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-30 08:00:05][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-30 08:00:05][INFO][1657463709.py:4] - ['response']:

What is a supercomputer? There is no place on Earth where they can make “a supercomputer.” It’s called Deep Space Computer, or DSC, and only the U.S. military has it.

What kind of computer can you build? DSC lets us take an image, an equation, a concept, and manipulate it in ways that are orders of magnitude more powerful than any human, and could one day outpace us. We use the computer to make scientific breakthroughs, and to make our everyday lives more complex and interesting.

What will the computer do? The computer’s job is to do mathematics, which means it can do a fraction of the number of calculations in a human lifetime. This does not mean that the computer has any physical characteristics. The computer’s computer is an intellectual tool, a computer that can understand the world, and it can learn how to solve problems. If you want to build a compu